# SNAP - NEXUSFORGE - Searching data

### Setup

The token can be found in the upper right corner on https://bbp.epfl.ch/nexus/web/ and pasted here

In [1]:
import getpass
from bluepysnap.nexus import NexusHelper

BUCKET = "nse/test"
TOKEN = getpass.getpass()
nexus = NexusHelper(BUCKET, TOKEN)

 ·······································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································

### Basic Searching

#### Getting a single record from Nexus

The most basic use case is getting a Nexus record based on it's ID that can be found on Nexus.

In [2]:
record = nexus.get_entity_by_id('https://bbp.epfl.ch/nexus/v1/resources/nse/test/_/20191025')
type(record)

bluepysnap.nexus.entity.Entity

`record` is a type of `Entity`. We'll cover that in the latter notebooks but for now, the important part is to know that 
* we found and retrieved the data from Nexus 
*`get_entity_by_id` returns a __single__ `Entity`.

#### Find resources based on their type

We can find data based on the data type. For, example, `DetailedCircuit`

In [3]:
records = nexus.get_entities('DetailedCircuit')
print(f'Length: {len(records)}\nType: {type(records)}\n')

Length: 4
Type: <class 'list'>



Here we can see that we found several records of `DetailedCircuit` and they were returned as a `list` (of `Entity` objects).

#### Filtering

`NexusHelper.get_entities` also accepts filters as an argument. These filters are passed in a dictionary. For example if we want to find a circuit based on its name __and__ creator we can do it like this:

In [4]:
filters = {
    "name": "Thalamus microcircuit v3",
    "createdBy": "ivaska"
}
records = nexus.get_entities('DetailedCircuit', filters)
print(f'Length: {len(records)}')

Length: 1


We may also create more complicated search filters that follow the data layout of objects in Nexus.
These are performed by having dictionaries inside dictionaries.
For example, to find a all circuits that have a brain region labeled as _Thalamus_ in Nexus:

In [5]:
filters = {
    "brainLocation": {
        "brainRegion": {
            "label": "Thalamus"
        }
    }
}
records = nexus.get_entities('DetailedCircuit', filters)
print(f'Length: {len(records)}')

Length: 4


#### Limiting search results
To limit the number of resources returned `nexusforge` provides a `limit` keyword:

In [6]:
records = nexus.get_entities('DetailedCircuit', limit=1)
print(f'Length: {len(records)}')

Length: 1


### Advanced Searching

#### "Look-back" searches

##### __DetailedCircuit.nodeCollection.memodelRelease.morphologyRelease.MorphologyRelease__

Above we covered "look-forward" search cases such as this (find `DetailedCircuit`s that are linked to a certain morphology release):

In [7]:
morphology_release_id = 'https://bbp.epfl.ch/nexus/v1/resources/nse/test/_/O1-20190624_morph_release'
filters = {
    "nodeCollection" : {
        "memodelRelease": {
            "morphologyRelease": {
                "id": morphology_release_id
            }
        }
    }
}
records = nexus.get_entities('DetailedCircuit', filters)
print(f'Length: {len(records)}')

Length: 1


Now, let's imagine we want to find morphology releases linked to circuits.
We can do that with a "look-back" operator `^`:

In [8]:
# This might be a bit difficult to understand but if we read it inside out, it's simpler
filters = {
    "^morphologyRelease": {               # (4) ...that that is linked to a morphology release.
        "^memodelRelease": {              # (3) ...that is linked to a ME model release...
            "^nodeCollection": {          # (2) ...is linked to a node collection...
                "type": "DetailedCircuit" # (1) Circuit...
            },
        },
    }}
records = nexus.get_entities('MorphologyRelease', filters) # (5) Get those morphology releases.
print(f'Length: {len(records)}')

Length: 3


##### __SimulationCampaign.used.DetailedCircuit__
Let's have another example. A "look-forward" search is for `SimulationCampaign`s that use a certain circuit:

In [9]:
circuit_id = 'https://bbp.epfl.ch/nexus/v1/resources/nse/test/_/O0-20180419'
filters = {
    "used": {
        "id": circuit_id
    }
}
records = nexus.get_entities('SimulationCampaign', filters)
print(f'Length: {len(records)}')

Length: 5


And similar "look-back" search would be finding the `DetailedCircuit` that is used by a certain simulation campaign (notice the `^` prefix in `used`):

In [10]:
simulation_campaign_id = 'https://bbp.epfl.ch/nexus/v1/resources/nse/test/_/ivaska.SimulationCampaignNexusTask_entity_managemen_https___bbp_epfl__gpfs_bbp_cscs_c_4b174cc572'
filters = {
    "^used": {                       # (2) is using (is linked to "used", that is linked to)...
        "id": simulation_campaign_id # (1) Something that has this id...
    }
}
records = nexus.get_entities('DetailedCircuit', filters) # (3) ...the DetailedCircuit we want to find.
print(f'Length: {len(records)}')

Length: 1
